In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


In [2]:
C2023II = pd.read_csv(r'C:\Users\Dussand\Desktop\proyectsDS\Python\Scrapping\delete_data\postulantes_por_carrera2023II.csv')
C2024I = pd.read_csv(r'C:\Users\Dussand\Desktop\proyectsDS\Python\Scrapping\delete_data\postulantes_por_carrera2024I.csv')
C2024II = pd.read_csv(r'C:\Users\Dussand\Desktop\proyectsDS\Python\Scrapping\delete_data\postulantes_por_carrera2024II.csv')
# C2025I = pd.read_csv(r'C:\Users\Dussand\Desktop\proyectsDS\Python\Scrapping\delete_data\postulantes_por_carrera2025I.csv')

In [3]:
#creamos una columna con el periodo del examen para cada periodo
C2023II['periodo'] = "2023II"
C2024I['periodo'] = "2024I"
C2024II['periodo'] = "2024II"
# C2025I['periodo'] = "2025I"



In [4]:
resultados_exam = pd.concat([C2023II, C2024I, C2024II], ignore_index = True)

In [5]:
resultados_exam.to_csv('resultados_consolidados.csv', index=False)

In [ ]:
#cambiamos el nombre de las columnas
columns = {
    'CODIGO':'student_id',
    'APELLIDOS Y NOMBRES':'full_name',
    'ESCUELA PROFESIONAL':'career_1',
    'PUNTAJE FINAL':'score',
    'MERITOE.P':'merit_order',
    'OBSERVACI&OacuteN':'observation'
}

C2023II.rename(columns=columns, inplace = True)

C2023II.info()

In [ ]:
columns = {
    'CODIGO':'student_id',
    'APELLIDOS Y NOMBRES':'full_name',
    'ESCUELA PROFESIONAL':'career_1',
    'PUNTAJE FINAL':'score',
    'MERITOE.P ALCANZA VACANTE':'merit_order',
    'OBSERVACI&OacuteN':'observation',
    'ESCUELA SEGUNDA OPCIÓN':'career_2'
}

C2024I.rename(columns=columns, inplace = True)

C2024I.info()

In [ ]:
columns = {
    'CODIGO':'student_id',
    'APELLIDOS Y NOMBRES':'full_name',
    'ESCUELA PROFESIONAL (PRIMERA OPCIÓN)':'career_1',
    'PUNTAJE':'score',
    'MERITOE.P':'merit_order',
    'OBSERVACI&OacuteN':'observation',
    'ESCUELA PROFESIONAL (SEGUNDA OPCIÓN)':'career_2'
}

C2024II.rename(columns=columns, inplace = True)

C2024II.info()

In [7]:
# #cambiamos el nombre de las columnas
# columns = {
#     'CODIGO':'student_id',
#     'APELLIDOS Y NOMBRES':'full_name',
#     'ESCUELA PROFESIONAL':'career_1',
#     'PUNTAJE':'score',
#     'MERITOE.P':'merit_order',
#     'OBSERVACI&OacuteN':'observation'
# }

# C2025I.rename(columns=columns, inplace = True)

# C2025I.info()

In [8]:
#anexamos los dataframes

resultados_exam = pd.concat([C2023II, C2024I, C2024II], ignore_index = True)

In [ ]:
resultados_exam.sample(10)

In [10]:
resultados_exam['location'] = resultados_exam['career_1'].str.extract(r' - (.+)')  # Extraer 'LIMA'
resultados_exam['career_1'] = resultados_exam['career_1'].str.replace(r' - .+', '', regex=True)  # Remover '- LIMA' de 'career_1'

In [ ]:
resultados_exam.sample(10)

In [ ]:
resultados_exam['periodo'].unique()

In [ ]:
resultados_exam['observation'].unique()

In [14]:
#corregimos los textos

resultados_exam['observation'] = resultados_exam['observation'].replace({
    'ALCANZO VACANTE SEGUNDA OPCIÃ\x93N':'ALCANZO VACANTE SEGUNDA OPCIÓN',
    'ALCANZO VACANTE PRIMERA OPCIÃ\x93N':'ALCANZO VACANTE PRIMERA OPCION',
    'ALCANZO VACANTE':'ALCANZO VACANTE PRIMERA OPCION'
})

In [ ]:
resultados_exam['observation'] = resultados_exam['observation'].fillna('NO ALCANZO VACANTE')
resultados_exam['observation'].unique()

In [ ]:
resultados_exam['location'] = resultados_exam['location'].fillna('LIMA')

resultados_exam['location'].unique()

In [ ]:
resultados_exam['career_2'].unique()

In [ ]:
#analizamos los tipos de datos y corregimos en base a las necesidades para el analisis
resultados_exam['student_id'] = resultados_exam['student_id'].astype(object)

resultados_exam.tail()


In [21]:
resultados_exam.to_csv('resultados_consolidados.csv', index=False)

In [20]:
# resultados_exam.to_csv('resultados examen consolidado', index=False)

## EXPLORACION DE LOS DATOS

¿Cuál es la distribución de puntajes entre los estudiantes de diferentes carreras?
Analiza si hay carreras con puntajes consistentemente más altos o más bajos.

¿Qué carrera tiene el puntaje más alto y quién es el estudiante asociado?
Identifica al estudiante con el puntaje más alto y la carrera correspondiente.

¿Cuántos estudiantes lograron alcanzar un puesto vacante en comparación con los que no lo hicieron?
Compara el número de estudiantes que tienen una observación de "ALCANZO VACANTE" frente a aquellos sin esta observación.

¿Cuál es el promedio de puntajes por carrera?
Calcula el promedio de puntajes para cada carrera para ver cómo se comparan entre sí.

¿Hay alguna correlación entre el puntaje y el orden de mérito?
Investiga si existe alguna relación entre las puntuaciones y el orden de mérito, aunque algunos valores sean NaN.

¿Qué porcentaje de estudiantes tiene un puntaje por encima de 600?
Determina cuántos estudiantes tienen un puntaje superior a 600 en relación al total.

¿Cuál es el período con la mayor cantidad de estudiantes y cuál tiene los puntajes más altos?
Analiza la cantidad de estudiantes en cada período y su rendimiento promedio.

¿Existen diferencias notables en puntajes entre las diferentes cohortes de estudiantes (2023II vs. 2024I vs. 2024II)?
Compara los puntajes de los estudiantes en las diferentes cohortes.

¿Qué carreras tienen más estudiantes sin puntaje registrado?
Identifica qué carreras tienen más estudiantes con puntajes NaN.

1. Distribucion de los puntajes de los estudiantes de las carreras

    Se creo una funcion que muestra un histograma con la distribucion de los puntajes por cada carrera y por perido que escogieron los postulantes como primera opcion.

In [102]:
# Definimos la función para analizar la distribución de puntajes por carrera
def hist_career(career, periodo):
    # Filtrar el DataFrame para obtener solo los datos de la carrera deseada
    career_data = resultados_exam[(resultados_exam['career_1'] == career) & (resultados_exam['periodo'] == periodo)]
    
    # Verificar si hay datos para la carrera especificada
    if not career_data.empty:
        # Crear un histograma de los puntajes
        plt.figure(figsize=(10, 6))
        sns.histplot(career_data['score'], bins=10, kde=True)  # Agregar KDE para suavizar la curva
        plt.title(f'DISTRIBUCIÓN DE PUNTAJES EN {career} - {periodo}')
        plt.xlabel('Puntaje')
        plt.ylabel('Frecuencia')
        plt.axvline(career_data['score'].max(), color='red', linestyle='--', label='Puntaje Máximo')
        plt.legend()
        plt.show()
        
        # Imprimir el puntaje máximo
        max_score = career_data['score'].max()
        min_score = career_data[career_data['observation'] == 'ALCANZO VACANTE PRIMERA OPCION']['score'].min()
    
        print(f'EL PUNTAJE MÁXIMO PARA {career} ES {max_score} Y EL PUNTAJE MINIMO ES {min_score} EN EL PERIODO {periodo}')
    else:
        print(f'No hay datos disponibles para la carrera {career}.')

In [ ]:
resultados_exam['career_1'].unique()

In [ ]:
hist_career('INGENIERÍA DE SISTEMAS', '2024II')  # Reemplaza con la carrera que deseas analizar

Qué carrera tiene el puntaje más alto y quién es el estudiante asociado?
Identifica al estudiante con el puntaje más alto y la carrera correspondiente.

In [ ]:
highest_score = resultados_exam.loc[resultados_exam.groupby('periodo')['score'].idxmax()]
highest_score[['full_name','career_1','score', 'periodo']]

¿Cuántos estudiantes lograron alcanzar un puesto vacante en comparación con los que no lo hicieron?
Compara el número de estudiantes que tienen una observación de "ALCANZO VACANTE" frente a aquellos sin esta observación.


In [ ]:
resultados_exam['observation'].unique()

In [107]:
vacant_reached = resultados_exam[
    (resultados_exam['observation'] == 'ALCANZO VACANTE PRIMERA OPCION') | (resultados_exam['observation'] == 'ALCANZO VACANTE SEGUNDA OPCIÓN') | (resultados_exam['observation'] == 'NO ALCANZO VACANTE')
]

In [ ]:
vacant_reached['observation'].unique()

In [ ]:
comparison = vacant_reached.groupby(['periodo','observation'])['observation'].count().reset_index(name = 'count')
comparison

In [ ]:
# Pivotar el DataFrame para tener 'observation' como columnas
pivot_comparison = comparison.pivot(index='periodo', columns='observation', values='count').fillna(0)
pivot_comparison

In [ ]:
# Crear el gráfico de barras apilado
pivot_comparison.plot(kind = 'bar')
# Configurar etiquetas y título
plt.title('Comparacion de ingresados y no ingresados por Periodo')
plt.xlabel('Periodo')
plt.ylabel('Count')
plt.legend(title='OBSEERVACIONES', bbox_to_anchor=(1.05, 1), loc='upper left')

¿Cuál es el maximo de puntaje por carrera?
Calcula el promedio de puntaje para cada carrera para ver cómo se comparan entre sí.

In [112]:
def mean_score(periodo):

    #filtramos por periodo
    periodo_filter = resultados_exam[resultados_exam['periodo'] == periodo]

    #agrupamos por carrera y mostramos el promedio
    max_score = periodo_filter.groupby('career_1')['score'].max().sort_values(ascending = False).reset_index()
    top10 = max_score.nlargest(20, 'score')
    #creamos un grafico de lineas
    plt.figure(figsize = (10,6))
    sns.lineplot(x = 'career_1', y = 'score', data=top10)
    plt.xlabel('Carrera')
    plt.ylabel('Puntaje maximo')
    plt.title(f'Puntaje maximo por carrera en el periodo {periodo}')
    plt.xticks(rotation = 90)
   


In [ ]:
mean_score("2025I")

¿Hay alguna correlación entre el puntaje y el orden de mérito?
Investiga si existe alguna relación entre las puntuaciones y el orden de mérito, aunque algunos valores sean NaN.


In [114]:
df_numerico = resultados_exam.select_dtypes(include = 'number')

In [ ]:
correlation = df_numerico.corr()
correlation

In [ ]:
sns.heatmap(correlation, annot = True, cmap = 'coolwarm', linewidths=0.5)

En primera instancia se puede observar que existe una ligera correlacion negativa entre ambas variables. Se podrá observar mejor con un grafico de regresion

In [ ]:
sns.lineplot(x= 'merit_order', y = 'score', data = resultados_exam)

Aqui se puede apreciar una tendencia negativa, ya que los orden de meritos mas cercanos al 0 tienden a tener puntajes mas altos a los que mas alejados. Lo cual es algo obvio, ya que los primeros puestos tienden a tener los puntajes mas altos para que puedan alcanzar una vacante.

¿Qué porcentaje de estudiantes por encima de 900 alcanzaron vacante?
Determina cuántos estudiantes con un puntaje superior a 900 alcanzaron una vacante

In [118]:
def score_900(periodo):
    score_period = resultados_exam[(resultados_exam['periodo'] == periodo) & (resultados_exam['score'] >= 900)] #filtramos por periodo y por puntajes mayores o iguales 900

    proportion = score_period.groupby('observation')['observation'].count().reset_index(name = 'count')  #agrupamos por observacion para contar cuantos postulantes alcanzaron vacantes
    proportion['proportion'] = proportion['count'] / proportion['count'].sum() * 100 # mostramos el porcentaje
    
    labels = proportion['observation'] #denominamos las etiquetas
    plt.pie(proportion['proportion'], labels=labels, autopct= '%1.1f%%') #mostramos los porcentajes en un grafico de pie
    plt.title(f"PROPORCION DE LOS POSTULANTES CON PUNTAJE MAYOR A 900 PUNTOS DEL {periodo}")


In [ ]:
score_900('2024II')

¿Cuál es el período con la mayor cantidad de estudiantes y cuál tiene los puntajes más altos?
Analiza la cantidad de estudiantes en cada período y su rendimiento promedio.

In [ ]:
period_high = resultados_exam.groupby('periodo').agg({'full_name':'nunique', 'score':'max'}).reset_index()
period_high

In [ ]:
plt.figure(figsize=(10, 6))
x = range(len(period_high['periodo']))

# Graficar las barras
plt.bar(x, period_high['full_name'], label='Amount students', color='skyblue')

# Graficar la línea de score
plt.plot(x, period_high['score'], color='orange', label='Línea de Score')

# Configuración del gráfico
plt.xticks(x, period_high['periodo'])
plt.xlabel('Periodo')
plt.ylabel('Score')
plt.title('Gráfico de Barras con Línea de Score')
plt.legend()


# Mostrar el gráfico
plt.show()

¿Existen diferencias notables en puntajes entre las diferentes cohortes de carreras (2023II vs. 2024I vs. 2024II)?
Compara los puntajes de las carreras en las diferentes cohortes.

In [122]:
def cohort_career(career):
    career_filtered = resultados_exam[(resultados_exam['career_1'] == career) & (resultados_exam['observation'] == 'ALCANZO VACANTE PRIMERA OPCION')]

    if not career_filtered.empty:
         cohort_students = career_filtered.pivot_table(index = 'career_1', columns='periodo', values='score', aggfunc='max')
         print(cohort_students)
         sns.heatmap(cohort_students, annot=True, fmt=".2f")
         plt.xlabel('Periodo')
         plt.ylabel('')
         plt.show()
    else:
        print('No se encontraron datos para la carrera seleccionada')


In [ ]:
cohort_career('ADMINISTRACIÓN DE NEGOCIOS INTERNACIONALES')


¿Qué carreras tienen más estudiantes sin puntaje registrado?
Identifica qué carreras tienen más estudiantes con puntajes NaN.


In [ ]:
#filtramos el dataframe con los postulantes que no ingresaron
sin_vacante = resultados_exam[(resultados_exam['observation'] == 'NO ALCANZO VACANTE')]
sin_vacante

In [ ]:
observation = sin_vacante.pivot_table(index='observation', columns='periodo', values='student_id', aggfunc='count').sort_values(by = 'observation',ascending=False).reset_index()
observation

In [ ]:
# Convertir a formato largo con melt
df_melt = observation.melt(id_vars='observation', var_name='periodo', value_name='cantidad')
df_melt

In [ ]:
labels = ['NO ALCANZO VACANTE']
sns.lineplot(x='periodo', y='cantidad', data=df_melt)
plt.legend(labels=labels)
plt.title('CANTIDAD DE POSTULANTES QUE NO ALCANZARON VACANTES POR PERIODO')

In [128]:
def wo_vacant(career):
    
    sin_vacante_filtered = sin_vacante[sin_vacante['career_1'] == career]

    if not sin_vacante_filtered.empty:
     wo_lineplot = sin_vacante_filtered.pivot_table(index = 'career_1', columns='periodo', values='student_id', aggfunc='count').reset_index()

     career_wo_vacant_melt = wo_lineplot.melt(id_vars= 'career_1', var_name='periodo', value_name='cantidad')
     sns.lineplot(x='periodo', y='cantidad', data=career_wo_vacant_melt, marker = 'o')
     plt.title(f'CANTIDAD DE POSTULANTES QUE NO ALCANZARON VACANTE DE {career}')
     for i in range(career_wo_vacant_melt.shape[0]):
        plt.text(career_wo_vacant_melt['periodo'][i], career_wo_vacant_melt['cantidad'][i], str(career_wo_vacant_melt['cantidad'][i]), 
             ha='center', va='bottom')

    else:
        print(f'No se encontraron datos para la carrera de {career}')
    
    return career_wo_vacant_melt


In [ ]:
wo_vacant('INGENIERÍA DE SOFTWARE')

In [ ]:
resultados_exam.head()

In [ ]:
resultados_exam.info()

In [132]:
areas_examen = [
    {'area_code': 'A', 'career_1': 'MEDICINA HUMANA'},
    {'area_code': 'A', 'career_1': 'OBSTETRICIA'},
    {'area_code': 'A', 'career_1': 'ENFERMERÍA'},
    {'area_code': 'A', 'career_1': 'TECNOLOGÍA MÉDICA'},
    {'area_code': 'A', 'career_1': 'LABORATORIO CLÍNICO Y ANATOMÍA PATOLÓGICA'},
    {'area_code': 'A', 'career_1': 'TERAPIA FÍSICA Y REHABILITACIÓN'},
    {'area_code': 'A', 'career_1': 'RADIOLOGÍA'},
    {'area_code': 'A', 'career_1': 'TERAPIA OCUPACIONAL'},
    {'area_code': 'A', 'career_1': 'NUTRICIÓN'},
    {'area_code': 'A', 'career_1': 'FARMACIA Y BIOQUÍMICA'},
    {'area_code': 'A', 'career_1': 'CIENCIAS DE LOS ALIMENTOS'},
    {'area_code': 'A', 'career_1': 'TOXICOLOGÍA'},
    {'area_code': 'A', 'career_1': 'ODONTOLOGÍA'},
    {'area_code': 'A', 'career_1': 'MEDICINA VETERINARIA'},
    {'area_code': 'A', 'career_1': 'PSICOLOGÍA'},
    {'area_code': 'A', 'career_1': 'PSICOLOGÍA ORGANIZACIONAL Y DE LA GESTIÓN HUMANA'},
    {'area_code': 'B', 'career_1': 'QUÍMICA'},
    {'area_code': 'B', 'career_1': 'CIENCIAS BIOLÓGICAS'},
    {'area_code': 'B', 'career_1': 'GENÉTICA Y BIOTECNOLOGÍA'},
    {'area_code': 'B', 'career_1': 'MICROBIOLOGÍA Y PARASITOLOGÍA'},
    {'area_code': 'B', 'career_1': 'FÍSICA'},
    {'area_code': 'B', 'career_1': 'MATEMÁTICA'},
    {'area_code': 'B', 'career_1': 'ESTADÍSTICA'},
    {'area_code': 'B', 'career_1': 'INVESTIGACIÓN OPERATIVA'},
    {'area_code': 'B', 'career_1': 'COMPUTACIÓN CIENTÍFICA'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA QUÍMICA'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA AGROINDUSTRIAL'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA MECÁNICA DE FLUIDOS'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA GEOLÓGICA'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA GEOGRÁFICA'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA DE MINAS'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA METALÚRGICA'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA CIVIL'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA AMBIENTAL'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA INDUSTRIAL'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA TEXTIL Y CONFECCIONES'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA DE SEGURIDAD Y SALUD EN EL TRABAJO'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA ELECTRÓNICA'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA ELÉCTRICA'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA DE TELECOMUNICACIONES'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA DE SISTEMAS'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA DE SOFTWARE'},
    {'area_code': 'E', 'career_1': 'DERECHO'},
    {'area_code': 'E', 'career_1': 'CIENCIA POLÍTICA'},
    {'area_code': 'E', 'career_1': 'LITERATURA'},
    {'area_code': 'E', 'career_1': 'FILOSOFÍA'},
    {'area_code': 'E', 'career_1': 'LINGÜÍSTICA'},
    {'area_code': 'E', 'career_1': 'COMUNICACIÓN SOCIAL'},
    {'area_code': 'E', 'career_1': 'ARTE'},
    {'area_code': 'E', 'career_1': 'BIBLIOTECOLOGÍA Y CIENCIAS DE LA INFORMACIÓN'},
    {'area_code': 'E', 'career_1': 'DANZA'},
    {'area_code': 'E', 'career_1': 'CONSERVACIÓN Y RESTAURACIÓN'},
    {'area_code': 'E', 'career_1': 'EDUCACIÓN'},
    {'area_code': 'E', 'career_1': 'EDUCACIÓN INICIAL'},
    {'area_code': 'E', 'career_1': 'EDUCACIÓN PRIMARIA'},
    {'area_code': 'E', 'career_1': 'EDUCACIÓN SECUNDARIA'},
    {'area_code': 'E', 'career_1': 'EDUCACIÓN FÍSICA'},
    {'area_code': 'E', 'career_1': 'HISTORIA'},
    {'area_code': 'E', 'career_1': 'SOCIOLOGÍA'},
    {'area_code': 'E', 'career_1': 'ANTROPOLOGÍA'},
    {'area_code': 'E', 'career_1': 'ARQUEOLOGÍA'},
    {'area_code': 'E', 'career_1': 'TRABAJO SOCIAL'},
    {'area_code': 'E', 'career_1': 'GEOGRAFÍA'},
    {'area_code': 'D', 'career_1': 'ADMINISTRACIÓN'},
    {'area_code': 'D', 'career_1': 'ADMINISTRACIÓN DE TURISMO'},
    {'area_code': 'D', 'career_1': 'ADMINISTRACIÓN DE NEGOCIOS INTERNACIONALES'},
    {'area_code': 'D', 'career_1': 'CONTABILIDAD'},
    {'area_code': 'D', 'career_1': 'GESTIÓN TRIBUTARIA'},
    {'area_code': 'D', 'career_1': 'AUDITORÍA EMPRESARIAL Y DEL SECTOR PÚBLICO'},
    {'area_code': 'D', 'career_1': 'ECONOMÍA'},
    {'area_code': 'D', 'career_1': 'ECONOMÍA PÚBLICA'},
    {'area_code': 'D', 'career_1': 'ECONOMÍA INTERNACIONAL'},
    {'area_code': 'A', 'career_1': 'TEC. MED. LAB. CLÍNICO Y ANATOMÍA PATOLÓGICA'},
    {'area_code': 'A', 'career_1': 'TEC. MED. TERAPIA FÍSICA Y REHABILITACIÓN'},
    {'area_code': 'A', 'career_1': 'TEC. MED. RADIOLOGÍA'},
    {'area_code': 'A', 'career_1': 'TEC. MED. TERAPIA OCUPACIONAL'},
    {'area_code': 'A', 'career_1': 'NUTRICION'},
    {'area_code': 'E', 'career_1': 'LINGUÍSTICA'},
    {'area_code': 'D', 'career_1': 'PRESUPUESTO Y FINANZAS PÚBLICAS'},
    {'area_code': 'C', 'career_1': 'INGENIERÍA BIOMÉDICA'},
    {'area_code': 'B', 'career_1': 'CIENCIAS DE LA COMPUTACIÓN'},
    {'area_code': 'E', 'career_1': 'LENGUAS, TRADUCCIÓN E INTERPRETACIÓN'},
    {'area_code': 'D', 'career_1': 'ADMINISTRACIÓN DE LA GASTRONOMÍA'},
    {'area_code': 'D', 'career_1': 'ADMINISTRACIÓN MARÍTIMA Y PORTUARIA'},
    {'area_code': 'D', 'career_1': 'MARKETING'},
    {'area_code':'C', 'career_1':'ARQUITECTURA Y URBANISMO'}

]

df_areas = pd.DataFrame(areas_examen)
# df_areas.to_csv('areas_sanmarcos', index = False)

In [ ]:
uniques = df_areas['career_1'].unique()
uniques

In [134]:
#separamos las ciudades de las carreras para un mejor analisis
#resultados_exam['location_2'] = resultados_exam['career_2'].str.extract(r' - (.+)')  # Extraer 'LIMA'
#resultados_exam['career_2'] = resultados_exam['career_2'].str.replace(r' - .+', '', regex=True)  # Remover '- LIMA' de 'career_1'

In [135]:
def segunda_carrera(carrera, periodo):
    
    segunda_opcion = resultados_exam[(resultados_exam['career_1'] == carrera) & (resultados_exam['periodo'] == periodo) & (resultados_exam['observation'] == 'ALCANZO VACANTE SEGUNDA OPCIÓN') ]  

    if not segunda_opcion.empty:
     heat_segundaopcion = segunda_opcion.pivot_table(index='career_2', columns='career_1', values = 'student_id', aggfunc='count').reset_index().sort_values(by = carrera, ascending = False)
     heat_segundaopcion

     plt.figure(figsize=(10,6))
     sns.barplot(x = 'career_2', y = carrera, data=heat_segundaopcion )
     plt.xticks(rotation = 90)
     plt.title(f'CANTIDAD DE ALUMNOS QUE ESCOGEN OTRA CARRERA COMO SEGUNDA OPCION DE LA CARRERA DE {carrera}')

    else:
        print(f'No se encontraron datos para la carrera de {carrera}')
    
    return heat_segundaopcion


In [ ]:
segunda_carrera('MEDICINA HUMANA', '2024II')

In [ ]:
segunda_opcion = resultados_exam[(resultados_exam['observation'] == 'ALCANZO VACANTE SEGUNDA OPCIÓN') ] 


second_choice = segunda_opcion.pivot_table(index='career_2', columns='career_1', values = 'student_id', aggfunc='count').fillna(0).reset_index()
second_choice.head()

In [ ]:
resultados_exam.columns

In [ ]:
df_merge = resultados_exam.merge(df_areas, on='career_1', how = 'outer')
df_merge.sample(5)

In [140]:
area_periodo_filtered = resultados_exam[resultados_exam['periodo'] == '2023I']

In [141]:
#mostramos un dataframe con el numero de postulantes y numero de ingresantes

competencia = df_merge.pivot_table(index='area_code', columns='observation', values ='student_id', aggfunc='count').fillna(0).reset_index()
competencia['total_estudiantes'] = competencia['ALCANZO VACANTE PRIMERA OPCION'] + competencia['ALCANZO VACANTE SEGUNDA OPCIÓN'] + competencia['ANULADO'] + competencia['AUSENTE'] + competencia['NO ALCANZO VACANTE']

In [142]:
competencia = competencia[
    ['area_code','ALCANZO VACANTE PRIMERA OPCION', 'ALCANZO VACANTE SEGUNDA OPCIÓN', 'total_estudiantes']
].dropna(axis = 1)

In [ ]:
competencia

In [ ]:
#cantidad de estudiantes por location y puntajes

score_location = resultados_exam.pivot_table(index='full_name', columns='periodo', values = 'student_id', aggfunc='count').fillna(0)
score_location['total_intentos'] = score_location['2023II'] + score_location['2024I'] + score_location['2024II'] + score_location['2025I']
score_location = score_location[score_location['total_intentos'] > 1]
score_location.sort_values(by = 'total_intentos', ascending=False)

mask = score_location.iloc[:,:-1].head() == 0

sns.heatmap(score_location.iloc[:, :-1].head(),
            linecolor='black',
            mask = mask)

In [ ]:
score_location.head()

In [ ]:
score_location_groupby = resultados_exam.groupby('location').agg({
    'score':'max',
    'student_id':'count'
}).fillna(0)

score_location_groupby

In [ ]:
sns.pairplot(resultados_exam)

In [148]:
ingresados = resultados_exam.pivot_table(
    index = 'periodo', columns='observation', values = 'student_id', aggfunc='count'
).fillna(0)

In [149]:
ingresados['total_students'] = ingresados['ALCANZO VACANTE PRIMERA OPCION'] + ingresados['ALCANZO VACANTE SEGUNDA OPCIÓN'] + ingresados['ANULADO'] + ingresados['AUSENTE'] + ingresados['NO ALCANZO VACANTE']


In [150]:
ingresados['PORCENTAJE'] = ingresados['ALCANZO VACANTE PRIMERA OPCION'] / ingresados['total_students'] * 100

In [ ]:
ingresados

In [152]:
funnel_df = resultados_exam.groupby('observation').agg({
    'student_id': 'count'
}).sort_values(by = 'student_id', ascending = False).reset_index()

In [ ]:
funnel_df

In [ ]:
funnel = resultados_exam.pivot_table(
    index = 'career_1',
    columns= 'observation',
    values= 'student_id',
    aggfunc='count'
).fillna(0).reset_index()

funnel['total_studiantes'] = (
        funnel.get('ALCANZO VACANTE PRIMERA OPCION', 0) 
      + funnel.get('ALCANZO VACANTE SEGUNDA OPCION', 0)
      + funnel.get('ANULADO', 0)
      + funnel.get('AUSENTE', 0)
      + funnel.get('NO ALCANZO VACANTE', 0)
)

funnel.head()

In [155]:
melted_df = pd.melt(funnel, 
                     id_vars=['career_1'], 
                     value_vars=['ALCANZO VACANTE PRIMERA OPCION', 'ALCANZO VACANTE SEGUNDA OPCIÓN', 'ANULADO', 'AUSENTE', 'NO ALCANZO VACANTE', 'total_studiantes'],
                     var_name='estado', 
                     value_name='cantidad')

In [ ]:
melted_df[melted_df['career_1'] == 'CONTABILIDAD']